In [1]:
import numpy as np
import pandas as pd

import paho.mqtt.client as mqtt

from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again.

The subscriber continuously receives the data. For each latest record(r) received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.

    Repeat this for 1000 times, and report the number of correct classifications.

    Question: "apply the 3NN classification to the last 5 records before r" is ambigous, is it from 0 - (len(list) - 5) or (len(list) - 5) - len(list)
    One intepretation would only ever test with the five latest records before r, the other would take all the records - the last 5 before r
    one: 5, the other is ever increasing, and ever more acurate, though also ever more computational expensive
    I must assume it would be the later, you want to train a model on new data in addition to what you already have to improve your model

In [2]:
def calculate_3nn_with_train_test(X, y):
    accuracy_row_list = []
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
        
    # Model intializing
    knn = KNeighborsClassifier(n_neighbors=3)

    # Training
    knn.fit(X_train, y_train)

    # Predicting on train and test set
    y_pred_train = knn.predict(X_train)
    y_pred_test = knn.predict(X_test)

    #print("KNN with k={}:".format(k))
        
    # Get the accuracy from metrics
    accuracy_train = accuracy_score(y_train, y_pred_train)
    #print("Training accuracy: {}".format(accuracy_train))
    accuracy_test = accuracy_score(y_test, y_pred_test)
    #print("Test accuracy: {}\r\n".format(accuracy_test))

    accuracy_row_list.append({"k": 3, "Training accuracy": accuracy_train, "Test accuracy": accuracy_test})

    return pd.DataFrame(accuracy_row_list)

In [3]:
def calculate_3nn(X, y):
    accuracy_row_list = []

    # Model intializing
    knn = KNeighborsClassifier(n_neighbors=3)

    # Training
    knn.fit(X, y)
    y_pred = knn.predict(X)
        
    # Get the accuracy from metrics
    accuracy_test = accuracy_score(y, y_pred)
    print("Test accuracy: {}\r\n".format(accuracy_test))

    correct_predictions = np.sum(y == y_pred)
    print("correct prediction count: {}\r\n".format(correct_predictions))

    accuracy_row_list.append({"k": 3, "Test accuracy": accuracy_test, "Number of correct classifications": correct_predictions})

    return pd.DataFrame(accuracy_row_list)

In [5]:
# This function defines what to do when we connect to the broker
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    # We subscribe to this particular data
    client.subscribe("myroom_super_S/S")

# This function defines what to do when we receives a message from the brokder
def on_message(client, userdata, msg):
    global my_list_s
    if(len(my_list_s.index) < 1000):
        temp_s = msg.payload.decode('utf-8')
        #print("Received myroom_super_S/S: ", temp_s)
        
        df_from_json = pd.read_json(StringIO(temp_s), orient='index')
    
        new_row = pd.DataFrame(df_from_json)
    
        my_list_s = pd.concat([my_list_s, new_row], ignore_index=True)
        
        if(len(my_list_s.index) > 5 and len(my_list_s.index) % 2 == 1): # train every 5 new data, could then probably just send the 5 data as one piece as well
            #print("My list: ", my_list_s)
    
            x = my_list_s[["Glucose", "BloodPressure", "Insulin"]]
            y = my_list_s["Outcome"]
            y = y.astype(int) # it was interpreted as a wrong type, it thought it was float but it is an int, 0 or 1
            
            result = calculate_3nn(x[x.index.max() - 5:(x.index.max())], y[x.index.max() - 5:(x.index.max())])
            global my_list_classification
            my_list_classification = pd.concat([my_list_classification, result], ignore_index=True)
            print(result)   
    else:
        global task_complete
        if(task_complete == -1):
            task_complete = 0

    if(task_complete == 0):
        task_complete = 1
        print("We are DONE")
        print()
        print("The result: ", my_list_classification)
        print("Total number of correct classifications: ", my_list_classification["Number of correct classifications"].max())
        best_row = my_list_classification[my_list_classification["Test accuracy"].max() == my_list_classification["Test accuracy"]]
        print("Best accuracy score: ", best_row)

# We create a client as the data subscriber and specify its actions for particular events
mqttc = mqtt.Client()
mqttc.on_connect = on_connect 
mqttc.on_message = on_message
my_list_s = pd.DataFrame(columns=["Glucose", "BloodPressure", "Insulin", "Outcome"])
my_list_classification = pd.DataFrame(columns=["k", "Test accuracy", "Number of correct classifications"])
task_complete = -1

# Now, we connect to the data broker.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# As a simple example, we just keep the data listening/receiving on and on...
mqttc.loop_forever()

Connected with result code 0


C:\Users\Bruger\AppData\Local\Temp\ipykernel_3664\2922684906.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  my_list_s = pd.concat([my_list_s, new_row], ignore_index=True)


Test accuracy: 0.8

correct prediction count: 4

   k  Test accuracy  Number of correct classifications
0  3            0.8                                  4
Test accuracy: 0.8

correct prediction count: 4

   k  Test accuracy  Number of correct classifications
0  3            0.8                                  4


C:\Users\Bruger\AppData\Local\Temp\ipykernel_3664\2922684906.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  my_list_classification = pd.concat([my_list_classification, result], ignore_index=True)


Test accuracy: 0.8

correct prediction count: 4

   k  Test accuracy  Number of correct classifications
0  3            0.8                                  4
Test accuracy: 1.0

correct prediction count: 5

   k  Test accuracy  Number of correct classifications
0  3            1.0                                  5
Test accuracy: 1.0

correct prediction count: 5

   k  Test accuracy  Number of correct classifications
0  3            1.0                                  5
Test accuracy: 0.8

correct prediction count: 4

   k  Test accuracy  Number of correct classifications
0  3            0.8                                  4
Test accuracy: 0.8

correct prediction count: 4

   k  Test accuracy  Number of correct classifications
0  3            0.8                                  4
Test accuracy: 1.0

correct prediction count: 5

   k  Test accuracy  Number of correct classifications
0  3            1.0                                  5
Test accuracy: 1.0

correct prediction count: 

KeyboardInterrupt: 